For Python programmers Ibis offers a way to write SQL in Python that allows for unit-testing, composability, and abstraction over specific query engines (e.g.BigQuery)! You can carry out joins, filters and other operations on your data in a familiar, Pandas-like syntax. Overall, using Ibis simplifies your workflows, makes you more productive, and keeps your code readable.

Let's start by getting ibis and sqlite installed on your system. In the next several cells, you will notice that the first character is an exclamation point: ***!*** This tells Jupyter that we're running a shell (terminal) command instead of python code in the notebook

## Install SQLite

In [ ]:
conda install -c anaconda sqlite 

## Install ibis

In [ ]:
! conda install -c conda-forge ibis-framework     

## Get the data
We're using the `Civic Art Collection` database, which can be downloaded here: https://data.sfgov.org/Culture-and-Recreation/Civic-Art-Collection/r7bn-7v9c using the `Export` service in the top right corner. Please select `CSV` for your export format. When you've completed the download, store the downloaded file in the `data` directory in this folder.

When you're done, you should see the following information when you list this folder:
>`data                       ibis_sqlite_tutorial.ipynb`

and when you list the Data folder you should see: 
>`Civic_Art_Collection.csv`
<br>

## Create the SQLite database

### let's first create variables for all of the things we're about to go do

In [ ]:
import os
path="data" #where did we store the data
file="Civic_Art_Collection.csv" #what's the name of the file we're importing
collection_name="civicArt.db" # what are we going to call the database
table_name="civicArtTable" # what are we going to call the table into which we are importing the file
path_separator=os.path.sep # get the path separator (forward or backward slash) for this operating system
conn=None # place holder for the db connection

### Create the SQL Collection (database)
SQLite wants a file with the name of the collection as the db. so we'll use the shell `touch` command to create it. 

In [ ]:
!touch "{collection_name}"

### Create the table schema
We will use pandas to help use create the schema and, later on, load the data into SQLite

In [ ]:
import pandas as pd
type_dic = {"object": "String", "float64": "Float64", "int64": "Int64"}
art_df = pd.read_csv(path+path_separator+file)
col_types = list(art_df.dtypes)
col_names = list(art_df.columns)
for idx, each in enumerate(col_names):
    col_names[idx] = "_".join(col_names[idx].split())
    col_types[idx] = type_dic[col_types[idx].__str__()]

### Create the table
This defines where the data will go and it's 'shape' - that is, the characteristics of each column of data

In [ ]:
import ibis
ibis.options.interactive = True
db = ibis.sqlite.connect(collection_name)
schema = ibis.Schema(names=col_names, types=col_types)
db.create_table(table_name, schema=schema)

### import the data into the table

In [ ]:
db.load_data(table_name, art_df, if_exists='replace')

#### list the tables

In [ ]:
sql_tables = db.list_tables()
print(sql_tables)

#### list the columns in the table

In [ ]:
# since the results of the preceding command is a list and we may get more than one table in the list
# print each table name and the columns for that table
for each in sql_tables:
    print(f"columns in {each}")
    print(f"{db.table(each).columns}")

## Querying 

Anything you can write in a SELECT statement you can write in Ibis. Let's test this out!
I’ll use the following code to find out which artists have art currently displayed in the city and what the title of their pieces are. 

### Selecting columns from a table 

In [ ]:
art = db.table(sql_tables[0])
sql_results = art["artist", "display_title"]
print(sql_results)

### Filtering Data

Next let's pick an artist and figure out where exactly all their art is located. Adriane Colburns’ display title, `Geological Ghost` caught my eye so let's choose them! 

I use the following commands to do this

In [ ]:
adrianes_art = art.filter(art["artist"] == 'Colburn, Adriane')
# adrianes_art["display_title", "street_address_or_intersection"]


In [ ]:
adrianes_art

So it turns out Adriane has two pieces on display, one at `4 Guy Place` and the other at `Dagget Street & 16th Street`. This is great, we already have some places we can add to our tourist itinerary!  

### Groupby

I don’t usually stay more than one or two days in a city after a conference, so it might be nice to know which locations have the most art on display. To figure this out we’ll use the following `groupby` expressions to get the information we need. 
We use `groupy` and `sort_by` to get the 10 locations in San Francisco with the most art! 

In [ ]:
art_loc = art.groupby("street_address_or_intersection").count('display_title')
most_art=art_loc.sort_by('display_title')
most_art[268:278]


#### Great, the first stop would be 1001 Potrero Avenue that has 59 titles on display!! 